In [14]:
import os
import nibabel as nib
import numpy as np

def file_to_ndarray(filepath):
    # Check the file extension
    _, file_extension = os.path.splitext(filepath)
    
    try:
        if file_extension in ['.nii', '.nii.gz']:  # Handle gzipped or regular NIfTI files
            # Load the NIfTI file
            nii_img = nib.load(filepath)
            # Convert to ndarray
            data = nii_img.get_fdata()
            #print(f"Loaded NIfTI file: {filepath}")
        else:
            print("Unsupported file format.")
            return None
        
        return data
    
    except Exception as e:
        print(f"An error occurred while processing the file: {e}")
        return None

In [26]:
def compute_histogram(ndarray, min_val=-4000, max_val=4000, bin_size=10):
    
    flat_array = ndarray.flatten()

    # Define the bin edges from -4000 to 4000 with a bin size of 10
    bins = np.arange(0, 1, 0.00125)  # 2001 to include the endpoint 2000 in the last bin

    # Compute histogram
    histogram_values, bin_edges = np.histogram(flat_array, bins=bins)

    # Convert histogram values to list
    histogram_list = histogram_values.tolist()

    return histogram_list, bin_edges


In [10]:
import csv

def append_to_csv(file_path, file_name, list_to_append):
    # Open the file in append mode

    list_to_append = [file_name] + list_to_append

    with open(file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        # Write the list as the last row in the CSV
        writer.writerow(list_to_append)
        #print("Successfully to the CSV.")


In [11]:
import pandas as pd

def read_csv_as_dataframe(filepath):
    # Read the CSV file without headers
    df = pd.read_csv(filepath, header=None)

    # Use the first column as the header
    headers = df.iloc[:, 0]  # Extract the first column as headers
    df = df.iloc[:, 1:]      # Remove the first column from the df
    df = df.T
    df.columns = headers     # Set the extracted column as headers

    return df

In [12]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_columns_of_dataframe(df, x_coords):
    # Check if the length of x_coords matches the number of rows in the DataFrame
    if len(x_coords) != len(df):
        raise ValueError("Length of x_coords must match the number of rows in the DataFrame")

    plt.figure(figsize=(10, 6))

    # Loop through each column in the DataFrame and plot
    for column in df.columns:
        plt.plot(x_coords, df[column], label=f'Column: {column}')

    plt.title('Voxels intensity distribution')
    plt.xlabel('Voxels intensity')
    plt.ylabel('Voxels count')
    #plt.legend()
    #plt.grid(True)
    plt.show()

In [28]:
import os
import tqdm

# Define the directory path where the .nii files are located
volume_folder = r"C:\Users\acer\Downloads\3_Preprocessed-20251127T013737Z-1-001\3_Preprocessed"  # Replace with your actual directory path
csv_path = rf"{volume_folder}\volxel_intensity_000125.csv"

nii_count = len([filename for filename in os.listdir(volume_folder) if filename.endswith('.nii.gz')])
print(f"There are {nii_count} .nii files in the {volume_folder}")

files = sorted(os.listdir(volume_folder))

# Loop through each file in the directory
error_log = []
for filename in tqdm.tqdm(files, desc = "Processing"):

    if filename.endswith('.nii.gz'):

        try:
            # Full path to the file
            file_path = os.path.join(volume_folder, filename)
            #print(f"Processing file: {filename}")

            # Read .nii file
            nii_img = nib.load(file_path)
            data = nii_img.get_fdata()

            # # Compute histogram
            histogram_values, bin_edges = compute_histogram(data, min_val=-4000, max_val=4000, bin_size=10)

            # # Write to csv
            append_to_csv(csv_path, filename, histogram_values)
        except Exception as e:
            print(f"Error processing {filename}: {e}")
            error_log.append(filename)

for file in error_log:
    print(file)


There are 407 .nii files in the C:\Users\acer\Downloads\3_Preprocessed-20251127T013737Z-1-001\3_Preprocessed


Processing: 100%|██████████| 415/415 [03:40<00:00,  1.88it/s]
